Figure des essaies à supprimer

- saccades dans GAP (début GAP - fin GAP + 150ms) > 45ms
- trop de saccade dans poursuite (>= 1/3 de la poursuite)

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import os
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from ANEMO import read_edf
from ANEMO import ANEMO
import numpy as np
from lmfit import  Model, Parameters
%matplotlib inline

In [2]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [3]:
Delete_list_trials = {}

t_sup = 280
b_s = 5
a_s = 15

opt_base = {'stop_search_misac': None, 'time_sup': t_sup,
            'before_sacc': b_s, 'after_sacc': a_s,
            } 

for x in range(len(sujet)) :
    
    timeStr = time[x]
    observer = sujet[x]
    mode = 'enregistrement'
    
    print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

    A = ANEMO(exp)
    
    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials = exp['N_trials']
    N_blocks = exp['N_blocks']
    
    Delete_list_trials['%s'%observer] = []
    for block in range(N_blocks) :
        supp_trial = []
        for trial in range(N_trials) :
            
            trial_data = trial + N_trials*block
            arg = A.arg(data[trial_data], trial=trial, block=block)
            opt = opt_base.copy()
            opt.update(arg)
            
            #velocity_NAN, new_sacc = A.velocity_NAN(**opt)
            velocity_NAN = A.velocity_NAN(**opt)[0]
            new_sacc = arg.saccades
            s_gap= []
            for s in range(len(new_sacc)) :
                s_gap.append([])
                #if new_sacc[s][1]-arg.t_0+a_s <= (len(arg.trackertime)) :
                for x_data in np.arange((new_sacc[s][0]-arg.t_0-b_s), (new_sacc[s][1]-arg.t_0+a_s)) :
                    if x_data in np.arange(arg.TargetOn-arg.t_0-150, arg.TargetOn-arg.t_0+150) :
                        s_gap[s].append(x_data)

            
            t = None
            
            # supp grosse saccade GAP
            for x in range(len(s_gap)) :
                if len(s_gap[x]) > 45 :
                    if t is None :
                        #print('GAP', block, trial)
                        supp_trial.append(trial)
                        t = True
            
            # supp trop saccade poursuite
            if t is None :
                lala2 = velocity_NAN[arg.TargetOn-arg.t_0:arg.TargetOff-arg.t_0-300]
                baba2 = [x for x in lala2 if str(x)=='nan']
                if len(baba2) >= len(lala2)/3.6 :
                    #print('poursuite', block, trial)
                    supp_trial.append(trial)
        
        print(block, supp_trial)
        Delete_list_trials['%s'%observer].extend([supp_trial])

AM 2017-10-23_100057
0 []
1 [134]
2 []
BMC 2017-09-26_095637
0 [76, 156]
1 [57, 66, 71, 126, 128, 129, 137]
2 [79, 137]
CS 2017-10-03_134421
0 []
1 []
2 [15, 95]
DC 2017-09-27_161040
0 [101, 120, 185]
1 [49, 66, 85, 127, 130, 144, 164]
2 [31, 36]
FM 2017-10-03_143803
0 [14, 29, 35, 48, 50, 58, 59, 60, 71, 73, 77, 80, 104, 157, 161, 167, 173, 174, 175, 178, 191, 198]
1 [12, 31, 64, 68, 69, 70, 96, 98, 121, 129, 194]
2 [100, 158, 177]
IP 2017-09-28_115250
0 []
1 [182]
2 []
LB 2017-09-20_151043
0 [10, 42, 82, 83, 90, 143, 150, 151, 163, 179]
1 [72, 116, 118, 123, 162, 164, 178]
2 [5, 11, 21, 32, 44, 53, 98]
OP 2017-10-26_121823
0 [28, 32, 38, 40, 46, 52, 68, 70, 71, 76, 81, 85, 86, 89, 90, 98, 122, 123, 126, 130, 131, 140, 143, 154, 166, 179, 183, 196, 199]
1 [0, 36, 48, 57, 83, 87, 94, 96, 105, 124, 136, 145, 149, 150, 154, 182, 185, 194, 197]
2 [1, 26, 44, 52, 72, 84, 91, 94, 95, 96, 97, 131, 137, 140, 145, 148, 157, 168, 173, 177, 186, 187]
RS 2017-11-08_094717
0 []
1 [30, 180]
2 [85, 

/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:329: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:329: RuntimeWarning: invalid value encountered in true_divide


0 [23, 50, 56, 96, 132, 137, 143, 148]
1 [21, 40]
2 [186, 199]
TN 2017-11-08_150410
0 [3, 4, 61, 117, 143, 160, 184]
1 [106, 126, 143]
2 [83, 87, 88, 89, 97]
YK 2017-11-17_172706
0 [47, 58, 62, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 78, 79, 84, 103, 112, 113, 114, 116, 117, 118, 123, 124, 127, 128, 132, 136, 137, 138, 141, 142, 143, 145, 147, 149, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 184, 185, 186, 189, 190]
1 [14, 17, 22, 23, 25, 30, 31, 32, 34, 36, 40, 41, 42, 43, 46, 53, 54, 65, 67, 69, 76, 77, 93, 107, 109, 112, 113, 133, 137, 138, 139, 156, 176, 192, 197]
2 [1, 3, 12, 26, 30, 69, 71, 81, 104, 105, 107, 108, 114, 117, 122, 144, 150, 172, 190, 192, 193, 197]


In [4]:
file = os.path.join('parametre', 'Delete_list_trials_velocity_fct.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(Delete_list_trials)

In [5]:
Bad_Fit = {}

Bad_Fit['AM'] = [[22], [], []]

Bad_Fit['BMC'] = [[], [129], [79, 118, 123]]

Bad_Fit['CS'] = [[69, 141, 162],
                 [35, 43, 44, 45, 47, 53, 58, 74, 119, 122, 123, 124, 127, 134, 135, 167],
                 [5, 8, 14, 15, 25, 90, 95, 176, 178, 193]]

Bad_Fit['DC'] = [[3, 9, 34, 55, 57, 66],
                 [7, 8, 10, 33, 123, 124, 192],
                 [16, 17, 18, 20, 59, 73]]

Bad_Fit['FM'] = [[29, 73, 104], [12, 70, 76], [78, 79, 116]]

Bad_Fit['IP'] = [[], [], []]

Bad_Fit['LB'] = [[29, 36, 50, 52, 94, 100, 163, 170, 176, 190, 196, 198],
                 [68, 72, 85, 91, 92, 132, 133, 135, 150, 155, 188],
                 [52, 95, 113, 146, 154, 163]]

Bad_Fit['OP'] = [[22, 28, 32, 42, 46, 53, 66, 68, 83, 90, 120, 131, 143, 157, 171, 199],
                 [16, 31, 36, 45, 56, 57, 62, 93, 99, 101, 103, 120, 122, 128, 131, 132, 135, 136,
                  137, 138, 141, 149, 152, 155, 167, 168, 180, 182, 186],
                 [5, 37, 44, 48, 53, 77, 84, 86, 87, 91, 93, 94, 95, 102, 104, 112, 114, 115, 126,
                  135, 136, 137, 139, 143, 144, 152, 159, 160, 168, 174, 190, 199]]

Bad_Fit['RS'] = [[34], [19, 55, 60, 137, 141, 177, 180], [39]]

Bad_Fit['SR'] = [[75, 96, 137, 143, 148],
                 [17, 39, 40, 56, 57, 61, 62, 63, 75, 79, 118, 128, 134, 137, 138, 141, 142, 161,
                  171, 175, 181, 182, 184],
                 [36, 91, 186, 190, 199]]

Bad_Fit['TN'] = [[0, 3, 4, 13, 25, 42, 45, 113, 117, 160, 187],
                 [45, 71, 72, 118, 126, 128, 152],
                 [56, 97, 138, 164]]

Bad_Fit['YK'] = [[66, 68, 71, 72, 73, 75, 76, 78, 79, 80, 82, 84, 85, 87, 92, 93, 113, 114,
                  117, 118, 123, 128, 130, 133, 136, 137, 138, 141, 142, 145, 147, 149, 153, 155, 156, 157,
                  158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
                  176, 180, 183, 185, 186, 187, 189, 190, 191, 195],
                 [11, 15, 17, 19, 22, 23, 25, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
                  45, 46, 54, 65, 67, 70, 71, 74, 76, 77, 81, 84, 87, 91, 92, 93, 94, 95, 96, 97, 99, 107, 109,
                  115, 126, 129, 132, 133, 134, 135, 137, 138, 139, 141, 142, 144, 145, 147, 148, 149, 151, 156,
                  157, 160, 169, 178, 180, 192, 193, 196, 198, 199],
                 [1, 2, 3, 4, 12, 16, 22, 25, 40, 44, 45, 47, 49, 50, 51, 53, 54, 58, 59, 60, 62, 65, 67, 68, 69,
                  70, 71, 73, 75, 76, 77, 78, 81, 87, 89, 91, 92, 94, 96, 97, 105, 106, 107, 121, 123, 125, 129,
                  131, 132, 135, 137, 139, 141, 143, 147, 148, 150, 158, 131, 172, 175, 180, 185, 191, 197, 198]]

In [6]:
file = os.path.join('parametre', 'Delete_list_Bad_trials_velocity.pkl')
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(Bad_Fit)

# Figure

In [7]:
def fct_fig(ax, data, exp, trial, N_trials, block, opt) :
    
    A = ANEMO(exp)
    Fit = ANEMO.Fit(exp)
    
    trial_data = trial + N_trials*block

    arg = A.arg(data[trial_data], trial=trial, block=block)
    opt = opt_base.copy()
    opt.update(arg)

    velocity_NAN = A.velocity_NAN(**opt)[0]

    #-------------------------------------------------
    # FIT
    #-------------------------------------------------
    old_latence, old_max, old_anti = A.classical_method.Full(velocity_NAN, arg.TargetOn-arg.t_0)
    f = Fit.Fit_trial(velocity_NAN, equation=equation, value_latence=old_latence, value_max=old_max, value_anti=old_anti, **opt)
    #-------------------------------------------------
    inde_v = Fit.generation_param_fit(equation=equation, **opt)[1]
    rv = f.values
    if 'do_whitening' in f.values.keys() :
        rv['do_whitening'] = False
    rv.update(inde_v)

    ax.set_ylabel('%s\nVelocity (°/s)'%(trial+1), fontsize=t_label)

    start = arg.TargetOn
    trackertime_s = arg.trackertime - start
    TargetOn_s = arg.TargetOn - start
    TargetOff_s = arg.TargetOff - start

    ax.axis([TargetOn_s-700, TargetOff_s+10, -39.5, 39.5])
    ax = A.Plot.deco(exp, ax, **opt)

    fit = ANEMO.Equation.fct_velocity(**rv)
    ax.plot(trackertime_s[:-time_sup], fit[:-time_sup], color='r', linewidth=2)
    ax.plot(trackertime_s, velocity_NAN, color='k', alpha=0.4)

    y = 0
    for name in f.values.keys() :
        if name in ['start_anti', 'latence'] :
            values = f.values[name] - (arg.TargetOn - arg.t_0)
            ax.bar(values, 80, bottom=-40, color='k', width=3, linewidth=0)
        else :
            values = f.values[name]

        ax.text((TargetOff_s-10), 35+y, "%s: %0.3f"%(name, values) , color='k', fontsize=t_text, va='center', ha='right')
        y = y - 5

    ax.set_xlabel('Time (ms)', fontsize=t_label)
    return ax

In [8]:
equation='fct_velocity'
time_sup = 280
t_label = 20
t_text = 14
fig_width = 12

opt_base = {'stop_search_misac': None, 'time_sup': 280,
            'param_fit': None, 'inde_vars': None,
            'step_fit':2, 'do_whitening': False,
            'before_sacc': 5, 'after_sacc': 15,
            't_label': t_label}    

In [9]:
for x in range(len(sujet)) :
    timeStr = time[x]
    observer = sujet[x]
    mode = 'enregistrement'
    
    print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials = exp['N_trials']
    N_blocks = exp['N_blocks']

    for block in range(N_blocks) :
        
        list_trial = Delete_list_trials['%s'%observer][block]
        print(list_trial)
        
        if len(list_trial) != 0 :
            fig_width= 12
            fig, axs = plt.subplots(len(list_trial), 1, figsize=(fig_width, (fig_width*(len(list_trial)/2))/1.6180))

            x = 0
            for trial in list_trial :

                if len(list_trial)==1 :
                    ax = axs
                else :
                    ax = axs[x]

                ax = fct_fig(ax, data, exp, trial, N_trials, block, opt)
        
                x=x+1
        
    
            plt.tight_layout()
            plt.subplots_adjust(hspace=0)

            plt.savefig('figures/suppresion/velocity/Delete_trials_velocity_%s_%s.pdf'%(observer, block+1))
            plt.close()

    print('FIN !!!')

AM 2017-10-23_100057
[]
[134]
[]
FIN !!!
BMC 2017-09-26_095637
[76, 156]
[57, 66, 71, 126, 128, 129, 137]
[79, 137]
FIN !!!
CS 2017-10-03_134421
[]
[]
[15, 95]
FIN !!!
DC 2017-09-27_161040
[101, 120, 185]
[49, 66, 85, 127, 130, 144, 164]


/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:547: RuntimeWarning: Mean of empty slice


[31, 36]
FIN !!!
FM 2017-10-03_143803
[14, 29, 35, 48, 50, 58, 59, 60, 71, 73, 77, 80, 104, 157, 161, 167, 173, 174, 175, 178, 191, 198]


/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:527: RuntimeWarning: Mean of empty slice


[12, 31, 64, 68, 69, 70, 96, 98, 121, 129, 194]
[100, 158, 177]
FIN !!!
IP 2017-09-28_115250
[]
[182]
[]
FIN !!!
LB 2017-09-20_151043
[10, 42, 82, 83, 90, 143, 150, 151, 163, 179]
[72, 116, 118, 123, 162, 164, 178]
[5, 11, 21, 32, 44, 53, 98]
FIN !!!
OP 2017-10-26_121823
[28, 32, 38, 40, 46, 52, 68, 70, 71, 76, 81, 85, 86, 89, 90, 98, 122, 123, 126, 130, 131, 140, 143, 154, 166, 179, 183, 196, 199]
[0, 36, 48, 57, 83, 87, 94, 96, 105, 124, 136, 145, 149, 150, 154, 182, 185, 194, 197]
[1, 26, 44, 52, 72, 84, 91, 94, 95, 96, 97, 131, 137, 140, 145, 148, 157, 168, 173, 177, 186, 187]
FIN !!!
RS 2017-11-08_094717
[]
[30, 180]
[85, 90, 170, 177, 190]
FIN !!!
SR 2017-11-16_153313
[23, 50, 56, 96, 132, 137, 143, 148]


/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:329: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:329: RuntimeWarning: invalid value encountered in true_divide


[21, 40]
[186, 199]
FIN !!!
TN 2017-11-08_150410
[3, 4, 61, 117, 143, 160, 184]
[106, 126, 143]
[83, 87, 88, 89, 97]
FIN !!!
YK 2017-11-17_172706
[47, 58, 62, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 78, 79, 84, 103, 112, 113, 114, 116, 117, 118, 123, 124, 127, 128, 132, 136, 137, 138, 141, 142, 143, 145, 147, 149, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 184, 185, 186, 189, 190]
[14, 17, 22, 23, 25, 30, 31, 32, 34, 36, 40, 41, 42, 43, 46, 53, 54, 65, 67, 69, 76, 77, 93, 107, 109, 112, 113, 133, 137, 138, 139, 156, 176, 192, 197]
[1, 3, 12, 26, 30, 69, 71, 81, 104, 105, 107, 108, 114, 117, 122, 144, 150, 172, 190, 192, 193, 197]
FIN !!!


In [10]:
for x in range(len(sujet)) :
    timeStr = time[x]
    observer = sujet[x]
    mode = 'enregistrement'
    
    print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials = exp['N_trials']
    N_blocks = exp['N_blocks']

    for block in range(N_blocks) :
        Bad_trial = Bad_Fit['%s'%observer][block]
        list_trial = Delete_list_trials['%s'%observer][block]
        print('init', Bad_trial)
        
        Bad_trial_0 = Bad_trial.copy()
        
        y = 0
        for x in range(len(Bad_trial_0)):
            if Bad_trial_0[x] in list_trial :    
                del Bad_trial[y]
                y = y-1
            y=y+1
        print('end', Bad_trial)
        
        if len(Bad_trial) != 0 :
            fig_width= 12
            fig, axs = plt.subplots(len(Bad_trial), 1, figsize=(fig_width, (fig_width*(len(Bad_trial)/2))/1.6180))

            x = 0
            for trial in Bad_trial :

                if len(Bad_trial)==1 :
                    ax = axs
                else :
                    ax = axs[x]

                ax = fct_fig(ax, data, exp, trial, N_trials, block, opt)
        
                x=x+1
        
    
            plt.tight_layout()
            plt.subplots_adjust(hspace=0)

            plt.savefig('figures/suppresion/velocity/Bad_trials_velocity_%s_%s.pdf'%(observer, block+1))
            plt.close()

    print('FIN !!!')

AM 2017-10-23_100057
init [22]
end [22]
init []
end []
init []
end []
FIN !!!
BMC 2017-09-26_095637
init []
end []
init [129]
end []
init [79, 118, 123]
end [118, 123]
FIN !!!
CS 2017-10-03_134421
init [69, 141, 162]
end [69, 141, 162]
init [35, 43, 44, 45, 47, 53, 58, 74, 119, 122, 123, 124, 127, 134, 135, 167]
end [35, 43, 44, 45, 47, 53, 58, 74, 119, 122, 123, 124, 127, 134, 135, 167]
init [5, 8, 14, 15, 25, 90, 95, 176, 178, 193]
end [5, 8, 14, 25, 90, 176, 178, 193]
FIN !!!
DC 2017-09-27_161040
init [3, 9, 34, 55, 57, 66]
end [3, 9, 34, 55, 57, 66]
init [7, 8, 10, 33, 123, 124, 192]
end [7, 8, 10, 33, 123, 124, 192]
init [16, 17, 18, 20, 59, 73]
end [16, 17, 18, 20, 59, 73]
FIN !!!
FM 2017-10-03_143803
init [29, 73, 104]
end []
init [12, 70, 76]
end [76]
init [78, 79, 116]
end [78, 79, 116]
FIN !!!
IP 2017-09-28_115250
init []
end []
init []
end []
init []
end []
FIN !!!
LB 2017-09-20_151043
init [29, 36, 50, 52, 94, 100, 163, 170, 176, 190, 196, 198]
end [29, 36, 50, 52, 94, 100,

/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:527: RuntimeWarning: Mean of empty slice


FIN !!!
